In [ ]:
#подготовка таблицы, добавление и удаление полей

import pandas as pd
import collections
from datetime import date, timedelta
import re
from itertools import chain
from xlrd import open_workbook
import numpy as np


table = pd.read_excel("~\Downloads\Telegram Desktop\db3.xlsx")
table1 = table.copy()

table.drop('Адрес', axis=1, inplace=True)
table.drop('ID', axis=1, inplace=True)


source = table['Source']
source[source == 'ngs'] = 'НГС'
source[source == 'avito'] = 'Avito'
source[source == 'kvadrat54'] = 'Kvadrat54.ru'
source[source == 'Skladoiskatel'] = 'Складоискатель'

table.drop('Source', axis=1, inplace=True)
table.drop('UniqueID', axis=1, inplace=True)


table.insert(0, 'Name', None)
table['Name'] = table['name']
table.drop('name', axis=1, inplace=True)
table = table.rename(index=str, columns={"Name": "name", "Street": "street", "House": "house", "City": "city"})

table['street'] = table['street'].astype(str)
table['house'] = table['house'].astype(str)



table.insert(1, 'coordinates', None)
table['coordinates'] = table['pos']
table.drop('pos', axis=1, inplace=True)

table.drop('свободный столбец (сохранение всех ссылок на повторы)', axis=1, inplace=True)


cols = table.columns.tolist()
new_cols = cols[:cols.index('operator_name')+1] #таблица
new_cols1 = cols[cols.index('operator_name')+1:] #листинги
table1 = table[new_cols1]
table = table[new_cols]


cols = table.columns.tolist()
cols.remove('listings')
cols.extend(['listings'])
table = table[cols]


table.to_excel('db.xlsx')

In [ ]:
#создание json-файла

book = open_workbook('db.xlsx')
sheet = book.sheet_by_index(0)

keys = [sheet.cell(0, col_index).value for col_index in range(sheet.ncols)]

dict_list = []
for row_index in range(1, sheet.nrows):
    d = collections.OrderedDict([(keys[col_index],sheet.cell(row_index, col_index).value )
                     for col_index in range(sheet.ncols)])
    dict_list.append(d)

table2 = table1.loc[:, ~table1.columns.str.contains('^Unnamed')]
links = table2.filter(regex=("List1_link"))
dates = table2.filter(regex=("date"))
prices = table2.filter(regex=("price"))

nrows, ncols = table1.shape
ncols = int(ncols/3)
unav = [np.nan, 'null', pd.NaT]
for row in range(nrows):
    listings = []
    for col in range(ncols):
        if (links.iloc[row,col] not in unav) and (dates.iloc[row,col] not in unav):
            date = np.nan
            sourc = np.nan
            price = np.nan
            if (dates.iloc[row,col] not in unav):
                date = '20'+dates.iloc[row,col].strftime('%y-%m-%d')+'T00:00:00.000Z'
            if (source[row] not in unav):    
                sourc = source[row]
            if (prices.iloc[row,col] not in unav):    
                price = prices.iloc[row, col]
            link = links.iloc[row,col]
            listings.append(collections.OrderedDict([("date", {"$date": date}), ("source", sourc),
                                                     ("price", price), ("link", link)]))
        else: 
            continue
    dict_list[row].popitem(last=False)        
    coord = dict_list[row]['coordinates'].split(' ')
    dict_list[row]['coordinates'] = [float(coord[1]), float(coord[0])]
    dict_list[row]['listings'] = listings
    dict_list[row]['latest_listing'] = 'null'
    if (dates.iloc[row,0] not in unav):
        dict_list[row]['latest_listing'] = {"$date": '20' + dates.iloc[row, 0].strftime('%y-%m-%d')+'T00:00:00.000Z'}
    dict_list[row].move_to_end('listings')
    
for entry in dict_list:
    for key in entry.keys():
        if (key == 'website') and (entry[key] not in unav) and ('http' not in str(entry[key])):    
            entry[key] = 'http://' + str(entry[key])
        if entry[key] == 'null':   
            entry[key] = np.nan
        if entry[key] == 'true':
            entry[key] = True
        if entry[key] == 'false':
            entry[key] == False    

In [ ]:
#выгрузка в файл
import json
import simplejson

h = json.dumps(dict_list, ensure_ascii=False)
js = json.loads(h, object_pairs_hook=collections.OrderedDict)
with open('data.json', 'w', encoding = 'utf-8') as outfile:
    simplejson.dump(js, outfile, indent=2, ignore_nan=True, ensure_ascii=False)